In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os

# Initialising MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def extract_frames(video_path, output_folder, start_time=31, duration=10):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    start_frame = int(start_time * fps)
    end_frame = start_frame + int(duration * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    os.makedirs(output_folder, exist_ok=True)
    
    frame_count = start_frame
    extracted_frames = []
    
    while cap.isOpened() and frame_count < end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            
            if 0.3 < left_ankle.x < 0.7 or 0.3 < right_ankle.x < 0.7:
                frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
                cv2.imwrite(frame_filename, frame)
                extracted_frames.append((frame_count, landmarks))
        
        frame_count += 1
    
    cap.release()
    return extracted_frames, fps, total_frames, start_frame

# Main execution for image extraction
video_path = "/Users/bharatheeshls/Downloads/Gait_Analysis/Pavment.MP4"
output_folder = "/Users/bharatheeshls/Gait_Pavment"

start_time = 43  # Starting from 43rd second
duration = 10  # Extracting 10 seconds of video

extracted_frames, fps, total_frames, start_frame = extract_frames(video_path, output_folder, start_time, duration)
print(f"Extracted {len(extracted_frames)} frames")

I0000 00:00:1722776201.762275 1867618 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Extracted 1053 frames


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from scipy import interpolate

# Initialising MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def determine_gait_phase(landmarks):
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
    right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
    left_toe = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]
    right_toe = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]
    left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL]
    right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL]
    
    if left_heel.y < right_heel.y:
        if left_heel.y < left_toe.y:
            return "Initial Contact"
        elif left_ankle.y > left_toe.y:
            return "Terminal Stance"
        else:
            return "Mid Stance"
    else:
        if right_heel.y < right_toe.y:
            return "Initial Contact"
        elif right_ankle.y > right_toe.y:
            return "Terminal Stance"
        else:
            return "Mid Stance"

def calculate_step_parameters(left_ankle, right_ankle):
    step_length = abs(left_ankle.z - right_ankle.z)
    step_width = abs(left_ankle.x - right_ankle.x)
    return step_length, step_width

def calculate_knee_angle(landmarks):
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
    
    vector1 = np.array([left_hip.x - left_knee.x, left_hip.y - left_knee.y, left_hip.z - left_knee.z])
    vector2 = np.array([left_ankle.x - left_knee.x, left_ankle.y - left_knee.y, left_ankle.z - left_knee.z])
    
    cosine_angle = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

def analyse_gait(input_folder):
    gait_phases = []
    step_lengths = []
    step_widths = []
    knee_angles = []
    step_times = []
    stride_times = []
    landmarks_data = []
    
    last_left_step_frame = None
    last_right_step_frame = None
    
    frame_files = sorted([f for f in os.listdir(input_folder) if f.startswith('frame_') and f.endswith('.jpg')],
                         key=lambda x: int(x.split('_')[1].split('.')[0]))
    
    for frame_file in frame_files:
        frame_count = int(frame_file.split('_')[1].split('.')[0])
        frame_path = os.path.join(input_folder, frame_file)
        
        image = cv2.imread(frame_path)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_image)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            landmarks_flat = [landmark.x for landmark in landmarks] + [landmark.y for landmark in landmarks] + [landmark.z for landmark in landmarks]
            landmarks_data.append(landmarks_flat)
            
            phase = determine_gait_phase(landmarks)
            gait_phases.append(phase)
            
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            
            step_length, step_width = calculate_step_parameters(left_ankle, right_ankle)
            step_lengths.append(step_length)
            step_widths.append(step_width)
            
            knee_angle = calculate_knee_angle(landmarks)
            knee_angles.append(knee_angle)
            
            if phase == "Initial Contact":
                if left_ankle.y < right_ankle.y:  # Left foot contact
                    if last_left_step_frame is not None:
                        step_time = (frame_count - last_left_step_frame) / fps
                        step_times.append(step_time)
                        if last_right_step_frame is not None:
                            stride_time = (frame_count - last_right_step_frame) / fps
                            stride_times.append(stride_time)
                    last_left_step_frame = frame_count
                else:  # Right foot contact
                    if last_right_step_frame is not None:
                        step_time = (frame_count - last_right_step_frame) / fps
                        step_times.append(step_time)
                        if last_left_step_frame is not None:
                            stride_time = (frame_count - last_left_step_frame) / fps
                            stride_times.append(stride_time)
                    last_right_step_frame = frame_count
    
    return gait_phases, step_lengths, step_widths, knee_angles, step_times, stride_times, landmarks_data

def analyse_gait_cycle(gait_phases, step_lengths, step_widths, knee_angles, step_times, stride_times):
    avg_step_length = np.mean(step_lengths)
    avg_step_width = np.mean(step_widths)
    avg_knee_angle = np.mean(knee_angles)
    avg_step_time = np.mean(step_times)
    avg_stride_time = np.mean(stride_times)
    
    return avg_step_length, avg_step_width, avg_knee_angle, avg_step_time, avg_stride_time

def visualise_gait_analysis(step_lengths, step_widths, knee_angles, step_times, stride_times, fps, output_directory):
    # Creating a time array for 2 seconds
    time = np.linspace(0, 2, num=int(2*fps))
    
    # Function to interpolate and resample data
    def resample_data(data):
        original_time = np.linspace(0, len(data)/fps, num=len(data))
        f = interpolate.interp1d(original_time, data, kind='linear', fill_value='extrapolate')
        return f(time)
    
    # Resample all data series
    step_lengths_resampled = resample_data(step_lengths)
    step_widths_resampled = resample_data(step_widths)
    knee_angles_resampled = resample_data(knee_angles)
    step_times_resampled = resample_data(step_times)
    stride_times_resampled = resample_data(stride_times)
    
    plt.figure(figsize=(15, 15))
    
    plt.subplot(5, 1, 1)
    plt.plot(time, step_lengths_resampled)
    plt.title("Step Length Over Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Step Length (m)")
    
    plt.subplot(5, 1, 2)
    plt.plot(time, step_widths_resampled)
    plt.title("Step Width Over Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Step Width (m)")
    
    plt.subplot(5, 1, 3)
    plt.plot(time, knee_angles_resampled)
    plt.title("Knee Angle Over Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Knee Angle (degrees)")
    
    plt.subplot(5, 1, 4)
    plt.plot(time, step_times_resampled)
    plt.title("Step Time Over Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Step Time (s)")
    
    plt.subplot(5, 1, 5)
    plt.plot(time, stride_times_resampled)
    plt.title("Stride Time Over Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Stride Time (s)")
    
    plt.tight_layout()
    
    svg_path = os.path.join(output_directory, "gait_analysis_results.svg")
    plt.savefig(svg_path, format='svg')
    print(f"Visualisation saved as '{svg_path}'")
    
    plt.close()

def train_gait_phase_model(landmarks_data, labels):
    X = np.array(landmarks_data)
    y = np.array(labels)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model accuracy on test set: {accuracy:.2f}")
    
    return model

def predict_gait_phase(model, landmarks):
    prediction = model.predict([landmarks])[0]
    return prediction

# Main execution for gait analysis and machine learning
input_folder = "/Users/bharatheeshls/Gait_Pavment"
data_output_directory = "/Users/bharatheeshls/Output_values"
os.makedirs(data_output_directory, exist_ok=True)

fps = 30  

gait_phases, step_lengths, step_widths, knee_angles, step_times, stride_times, landmarks_data = analyse_gait(input_folder)
avg_step_length, avg_step_width, avg_knee_angle, avg_step_time, avg_stride_time = analyse_gait_cycle(gait_phases, step_lengths, step_widths, knee_angles, step_times, stride_times)

print("Gait Cycle Analysis:")
print(f"Average Step Length: {avg_step_length:.2f} m")
print(f"Average Step Width: {avg_step_width:.2f} m")
print(f"Average Knee Angle: {avg_knee_angle:.2f} degrees")
print(f"Average Step Time: {avg_step_time:.2f} s")
print(f"Average Stride Time: {avg_stride_time:.2f} s")
print(f"Frames used for calculations: {len(gait_phases)}")

visualise_gait_analysis(step_lengths, step_widths, knee_angles, step_times, stride_times, fps, data_output_directory)

# Debug information
print("\nDebug Information:")
print(f"Number of frames: {len(gait_phases)}")
print(f"Step lengths: {len(step_lengths)}")
print(f"Step widths: {len(step_widths)}")
print(f"Knee angles: {len(knee_angles)}")
print(f"Step times: {len(step_times)}")
print(f"Stride times: {len(stride_times)}")

# Saving results to CSV
csv_path = os.path.join(data_output_directory, 'gait_analysis_results.csv')
try:
    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Frame", "Step Length (m)", "Step Width (m)", "Knee Angle (degrees)", "Step Time (s)", "Stride Time (s)"])
        frame_files = sorted([f for f in os.listdir(input_folder) if f.startswith('frame_') and f.endswith('.jpg')])
        
        for i, frame_file in enumerate(frame_files):
            frame_count = int(frame_file.split('_')[1].split('.')[0])
            row = [frame_count]
            row.append(step_lengths[i] if i < len(step_lengths) else "")
            row.append(step_widths[i] if i < len(step_widths) else "")
            row.append(knee_angles[i] if i < len(knee_angles) else "")
            row.append(step_times[i] if i < len(step_times) else "")
            row.append(stride_times[i] if i < len(stride_times) else "")
            writer.writerow(row)

    print(f"Results saved to '{csv_path}'")
except Exception as e:
    print(f"An error occurred while writing to CSV: {e}")
    print(f"Error at frame index: {i}")

# Training and implementing machine learning model for gait phase detection
print("\n--- Gait Phase Prediction Model ---")
gait_phase_model = train_gait_phase_model(landmarks_data, gait_phases)

# Using the trained model to predict gait phases
predicted_gait_phases = [predict_gait_phase(gait_phase_model, landmarks) for landmarks in landmarks_data]

print("\nGait phase prediction completed.")

# Calculating and printing accuracy on the entire dataset
all_predictions = gait_phase_model.predict(landmarks_data)
overall_accuracy = accuracy_score(gait_phases, all_predictions)
print(f"\nOverall model accuracy on all data: {overall_accuracy:.2f}")

# Printing a summary of predicted phases
unique, counts = np.unique(predicted_gait_phases, return_counts=True)
print("\nPredicted Gait Phases Summary:")
for phase, count in zip(unique, counts):
    print(f"{phase}: {count} frames")

I0000 00:00:1723912835.303465 2937392 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Gait Cycle Analysis:
Average Step Length: 0.05 m
Average Step Width: 0.01 m
Average Knee Angle: 133.79 degrees
Average Step Time: 0.55 s
Average Stride Time: 0.74 s
Frames used for calculations: 66
Visualisation saved as '/Users/bharatheeshls/Output_values/gait_analysis_results.svg'

Debug Information:
Number of frames: 66
Step lengths: 66
Step widths: 66
Knee angles: 66
Step times: 51
Stride times: 50
Results saved to '/Users/bharatheeshls/Output_values/gait_analysis_results.csv'

--- Gait Phase Prediction Model ---
Model accuracy on test set: 0.93

Gait phase prediction completed.

Overall model accuracy on all data: 0.98

Predicted Gait Phases Summary:
Initial Contact: 52 frames
Mid Stance: 8 frames
Terminal Stance: 6 frames
